In [10]:
# DEBUG: Test whether CPU/GPU is being used
'''
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')
'''

'\nfrom theano import function, config, shared, sandbox\nimport theano.tensor as T\nimport numpy\nimport time\n\nvlen = 10 * 30 * 768  # 10 x #cores x # threads per core\niters = 1000\n\nrng = numpy.random.RandomState(22)\nx = shared(numpy.asarray(rng.rand(vlen), config.floatX))\nf = function([], T.exp(x))\nprint(f.maker.fgraph.toposort())\nt0 = time.time()\nfor i in range(iters):\n    r = f()\nt1 = time.time()\nprint("Looping %d times took %f seconds" % (iters, t1 - t0))\nprint("Result is %s" % (r,))\nif numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):\n    print(\'Used the cpu\')\nelse:\n    print(\'Used the gpu\')\n'

In [11]:
import numpy as np
import pickle as pk
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import mean_squared_error
# In 'th' mode, the channels dimension (the depth) is at index 1
K.set_image_dim_ordering('th') 
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X = None
y = None

try:  # X_train
    with open("value_net_x_input.pickle","rb") as pickle_in:
        X = np.array(pk.load(pickle_in))
except IOError:
    print("IOError", file=sys.stderr)
except EOFError:
    print("EOFError", file=sys.stderr)
    
try:  # y_train
    with open("value_net_y_input.pickle","rb") as pickle_in:
        y = np.array(pk.load(pickle_in))
except IOError:
    print("IOError", file=sys.stderr)
except EOFError:
    print("EOFError", file=sys.stderr)

#(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("X.shape", X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape", X_train.shape)

X.shape (302, 3, 10, 10)
X_train.shape (241, 3, 10, 10)


In [12]:
# reshape to be [samples][pixels/depth/channels][width][height]
X_train = X_train.reshape(X_train.shape[0], 3, 10, 10).astype('float32')
# X = X.reshape(X.shape[0], 3, 10, 10).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 3, 10, 10).astype('float32')

# normalize inputs from 0-255 to 0-1
# X_train = X_train / 255.0
# X_test = X_test / 255.0

# one hot encode outputs
# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)

'''
We can create Keras models and evaluate them with scikit-learn by using handy wrapper objects 
provided by the Keras library. This is desirable, because scikit-learn excels at evaluating models 
and will allow us to use powerful data preparation and model evaluation schemes with very few lines of code.
'''

# The Keras wrappers require a function as an argument. 
# This function that we must define is responsible for creating the neural network model to be evaluated.
def baseline_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, 10, 10), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, input_dim=100, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    
    # Compile model
    epochs = 20
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


# estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)
# build the model
model = baseline_model()
model.summary()
# Fit the model

model.fit(X_train, y_train, validation_data=(X_test, y_test), shuffle=True, nb_epoch=20, batch_size=32, verbose=1)
# Final evaluation of the model
score = model.evaluate(X_test, y_test, verbose=1)
print("Final MSE score", score)
# score = mean_squared_error(y_test, model.predict(X_test))

# kfold = KFold(X_train.shape[0], n_folds=10, random_state=seed)
# score = cross_val_score(estimator, X_train, y_train, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (score.mean(), score.std()))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 32, 10, 10)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 32, 10, 10)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 32, 10, 10)    9248        dropout_3[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 32, 5, 5)      0           convolution2d_4[0][0]            
___________________________________________________________________________________________

In [21]:
# DEBUG: For visualizing the difference between y_test and model.predict(X_test)

x_axis = np.arange(61)
s1 = y_test
s2 = model.predict(X_test, verbose=1)

fig = plt.figure(figsize=(11, 6))
plt.plot(x_axis, s1)
plt.plot(x_axis, s2)
plt.legend(['actual labels', 'predicted labels'], loc='upper left')
ax = fig.gca()
ax.set_xticks(numpy.arange(0,61, 1))
ax.set_yticks(numpy.arange(0,1.,0.1))
plt.grid()
plt.show()

print("Actual labels: ", s1)
print("Predicted labels: ", s2)


32/61 [==============>...............] - ETA: 0s

NameError: name 'numpy' is not defined